In [48]:
import tensorflow as tf
import pandas as pd
import numpy as np
import gzip
import simplejson
import json
import time
import scipy.spatial

In [122]:
class DataImporter(object):
    def __init__(self, path_to_data):
        # get the raw data
        raw_data = [json.loads(i) for i in gzip.open(path_to_data, "rt")]
#         raw_data = raw_data[:1000]
        self.data = pd.DataFrame(raw_data)
        self.num_reviews = len(self.data)
        
        # get the counts of users and products
        self.reviewers = self.data["reviewerID"].unique()
        self.num_reviewers = len(self.reviewers)
        self.products = self.data["asin"].unique()
        self.num_products = len(self.products)
        
        # create int-to-name dictionaries for each
        self.reviewer_to_num = {reviewer.lower(): idx for idx, reviewer in enumerate(self.reviewers)}
        self.num_to_reviewer = {self.reviewer_to_num[reviewer]: reviewer for reviewer in self.reviewer_to_num}
        self.product_to_num = {product.lower(): idx for idx, product in enumerate(self.products)}
        self.num_to_product = {self.product_to_num[product]: product for product in self.product_to_num}
        
    def create_train_test_split(self, frac_test):
        # get test data indices
        test_indices = np.random.choice(self.num_reviews,
                                         size=int(self.num_reviews * frac_test),
                                         replace=False)
        
        # split raw data into train/test
        raw_test = self.data.iloc[test_indices, :]
        raw_train = self.data.drop(test_indices).dropna()
        
        # get dimensions of the matrices
        dim = (self.num_reviewers, self.num_products)
        self.test_matrix = self.populate_user_product_review_matrix(dim, raw_test)
        self.train_matrix = self.populate_user_product_review_matrix(dim, raw_train)
        
        self.cos_sim_mat = np.zeros((self.num_products, self.num_products))
        for root_prod_idx, prod_ratings in enumerate(self.train_matrix.T):
            for comp_prod_idx, comp_ratings in enumerate(self.train_matrix.T):
                if comp_prod_idx >= root_prod_idx:
                    self.cos_sim_mat[root_prod_idx, comp_prod_idx] = 1 - scipy.spatial.distance.cosine(prod_ratings,
                                                                                                       comp_ratings)
        
        self.test = self.permute_matrix(self.test_matrix)
        self.train = self.permute_matrix(self.train_matrix)
        
    def permute_matrix(self, mat):
        perm = []
        for u_idx, row in enumerate(mat):
            for prod_idx, score in enumerate(row):
                if score > 0.0:
                    user_cpy = row.copy()
                    score = user_cpy[prod_idx]
                    user_cpy[prod_idx] = 0
                    prod_cpy = mat[:, prod_idx].copy()
                    other_movies_reviewed = np.where(user_cpy > 0)[0]
                    similarities = [cos_sim for cos_sim in self.cos_sim_mat[prod_idx, other_movies_reviewed]]
                    prod_cpy[u_idx] = 0
                    perm.append({"user": user_cpy,
                                 "reviewerID": self.num_to_reviewer[u_idx],
                                 "product": prod_cpy,
                                 "score": score,
                                 "asin": self.num_to_product[prod_idx],
                                 "cosine_similarity": np.mean(similarities)
                                 })
        return perm
    
    def populate_user_product_review_matrix(self, dimensions, dataset):
        assert isinstance(dataset, pd.DataFrame)
        ret_matrix = np.zeros(dimensions)
        for idx, row in dataset.iterrows():
            ret_matrix[self.reviewer_to_num[row["reviewerID"].lower()], self.product_to_num[row["asin"].lower()]] = row["overall"]
        return ret_matrix

In [125]:
data_importer = DataImporter("data/reviews_Amazon_Instant_Video_5.json.gz")
start = time.time()
data_importer.create_train_test_split(0.1)
end = time.time()

/home/michael/Documents/deep_learning/venv/lib/python3.6/site-packages/scipy/spatial/distance.py:644: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)
/home/michael/Documents/deep_learning/venv/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2909: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/michael/Documents/deep_learning/venv/lib/python3.6/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [223]:
class wide_and_deep(object):
    
    def __init__(self, train, test, num_users, num_prods):
        self.num_users = num_users
        self.num_products = num_prods
        self.train = np.copy(train)
        self.test = np.copy(test)
        
        np.random.shuffle(self.train)
        np.random.shuffle(self.test)
        
        self.num_examples = self.train.shape[0] + self.test.shape[0]
        self.perc_test = self.test.shape[0] / self.train.shape[0]
        
        self.train = self.add_all_features_columns(self.train)
        self.test = self.add_all_features_columns(self.test)
        
    def add_all_features_columns(self, dataset):
        dataset = np.array(list(map(self.add_num_ratings, dataset)) )
        dataset = np.array(list(map(self.add_top_ratings, dataset)) )
        dataset = np.array(list(map(self.add_bottom_ratings, dataset)) )
        dataset = np.array(list(map(self.add_average_ratings, dataset)) )
        dataset = np.array(list(map(self.add_percent_rating, dataset)) )
        
        dataset = pd.DataFrame(list(dataset))
        return dataset
        
    def add_num_ratings(self, data):
        data['num_user_ratings'] = np.count_nonzero(data['user'])
        data['num_movie_ratings'] = np.count_nonzero(data['product'])
        return data
        
    def add_top_ratings(self, data):
        data['top_user_rating'] = np.amax(data['user'])
        data['top_movie_rating'] = np.amax(data['product'])
        return data
    
    def add_bottom_ratings(self, data):
        data['bottom_user_rating'] = np.amin(data['user'])
        data['bottom_movie_rating'] = np.amin(data['product'])
        return data
    
    def add_average_ratings(self, data):
        data['average_user_rating'] = np.average(data['user'])
        data['average_movie_rating'] = np.average(data['product'])
        return data
    
    def add_percent_rating(self, data):
        l = len(np.where(data['user'] > 0.0)[0])
        if l > 0:
            data['percent_one_star'] = len(np.where(data['user'] == 1.0)[0])/l
            data['percent_two_star'] = len(np.where(data['user'] == 2.0)[0])/l
            data['percent_three_star'] = len(np.where(data['user'] == 3.0)[0])/l
            data['percent_four_star'] = len(np.where(data['user'] == 4.0)[0])/l
            data['percent_five_star'] = len(np.where(data['user'] == 5.0)[0])/l
        return data
        
    def total_ratings(self, data):
        count = 0.0
        for i in data:
            if i['score'] != 0.0:
                count += 1
        print(count)
        
    def build_model(self, deep_layers, deep_layer_activation, wide_output_dim, wide_activation):
        
        D = self.train.drop(["reviewerID", "asin", "score", "product", "user"], axis=1).shape[1] 
        
        with tf.name_scope("input"):
            X = tf.placeholder(tf.float32, [None, D], name="x_input")
        
        with tf.name_scope("input_normalization"):
            X = tf.layers.batch_normalization(X)
        
        T = lambda x: tf.matmul(tf.reshape(x, [x.shape[0], 1]), tf.reshape(x, [1, x.shape[0]]))
        cross_prod_mat = tf.map_fn(T, X)
        cross_prod = tf.map_fn(lambda x: tf.reshape(x, [-1]), cross_prod_mat)
        
        # wide part
        wide = tf.layers.dense(cross_prod, wide_output_dim, activation=wide_activation, use_bias=False)
        
        deep = X
        
        # deep part
        for i in deep_layers:
            deep = tf.layers.dense(deep, i, activation=deep_layer_activation, use_bias=False)
            
        # combine the output of wide and deep
        wide_and_deep = tf.concat([wide, deep], 1)
        logits = tf.layers.dense(wide_and_deep, 5, name="logits")
        
    def add_optimizer(self, learning_rate):
        
        default_graph = tf.get_default_graph()
        logits = default_graph.get_tensor_by_name("logits/BiasAdd:0")
        
        with tf.name_scope("optimizer_input"):
            global_step = tf.placeholder(tf.int64, [], name="global_step")
            y = tf.placeholder(dtype=tf.int64, shape=[None], name="y_input")
        
        with tf.name_scope("cross_entropy"):
            xentropy = tf.nn.softmax_cross_entropy_with_logits(labels=tf.one_hot(y, 5), logits=logits)
            with tf.name_scope("loss"):
                loss = tf.reduce_mean(xentropy, name="loss")
        
        with tf.name_scope("softmax"):
            # Softmax layer
            softmax = tf.nn.softmax(logits)
            # Predict train
            y_hat = tf.argmax(softmax, axis=1, name="y_hat")
            
        with tf.name_scope('train'):
            learning_rate = tf.train.exponential_decay(learning_rate, global_step, 1000, 0.96, staircase=True)
            optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
            training_op = optimizer.minimize(xentropy, name="training_op")
            
        with tf.name_scope("accuracy"):
            # Train Evaluation
            train_correct = tf.equal(y, y_hat)
            tf.reduce_mean(tf.cast(train_correct, tf.float64), name="acc")
            
    def train_model(self, sess, n_epochs, batch_size):
        
        # New***
        init = tf.global_variables_initializer() 

        # get default graph
        default_graph = tf.get_default_graph()
        
        # Get individual graph tensors
        X = default_graph.get_tensor_by_name("input/x_input:0")
        Y = default_graph.get_tensor_by_name("optimizer_input/y_input:0")
        global_step = default_graph.get_tensor_by_name("optimizer_input/global_step:0")
        
        # Get specific operations in graph
        training_op = default_graph.get_operation_by_name("train/training_op")
        accuracy = default_graph.get_tensor_by_name("accuracy/acc:0")
        loss = default_graph.get_tensor_by_name("cross_entropy/loss/loss:0")
        y_hat = default_graph.get_tensor_by_name("softmax/y_hat:0")
        
        # Create logdir
        now = time.strftime("%Y%m%d%H%M%S")
        root_logdir = "tf_logs"
        logdir = f"{root_logdir}/run"
        
        # Create Tensorboard file_writers
        summary_writer = tf.summary.FileWriter(logdir, default_graph)
        
        # Create Tensorboard Summaries 
        acc_train_summary = tf.summary.scalar("train_accuracy", accuracy)
        acc_test_summary = tf.summary.scalar("test_accuracy", accuracy)
        loss_summary = tf.summary.scalar('train_loss', loss)
        
        # Create Tensorboard histograms
        conv_histograms = []
        for i in tf.global_variables():
            if i.name[:6] == "conv2d":
                # need to switch colon with underscore because colons aren't allowed in summary names
                conv_histograms.append(tf.summary.histogram(i.name.replace(":", "_"), i))
        
        # Wrapper tells Tensorboard to assume everything is within same session
        sess.run(init)

        step = 0
        for epoch in range(1, n_epochs + 1):
            n_iters = int((self.num_examples * (1 - self.perc_test)) / batch_size)
            print_interval = 100
            best_test = 0.

            print(f"\nStarting epoch {epoch}, running for {n_iters} iterations")
            print(f"Printing evaluation metrics every {print_interval} iterations\n")

            for iteration in range(1, n_iters):
                step += 1
                indices = np.random.choice(int(self.num_examples * (1 - self.perc_test)), batch_size)
                X_batch = self.train.iloc[indices, :]
                user_series = X_batch["user"].apply(pd.Series)
                prod_series = X_batch["product"].apply(pd.Series)
                x_numerical_cols = X_batch.drop(["reviewerID", "asin", "score", "product", "user"], axis=1)
                y_batch = self.train.iloc[indices, :]["score"].as_matrix() - 1
                sess.run(training_op, feed_dict={X: x_numerical_cols,
                                                 Y: y_batch,
                                                 global_step: step})

                if iteration % print_interval == 0:
                    acc_train, train_summary, y_pred = sess.run([accuracy, tf.summary.merge([loss_summary, acc_train_summary]), y_hat],
                                                        {X: x_numerical_cols, Y: y_batch})
                    
                    test_x = self.test.drop(["reviewerID", "asin", "score", "product", "user"], axis=1).as_matrix()
                    test_y = self.test["score"].as_matrix()
                    acc_test, test_summary = sess.run([accuracy, acc_test_summary], {X: test_x,
                                                                                     Y: test_y - 1})

                    # Add batch train loss & accuracy to Tensorboard
                    summary_writer.add_summary(tf.summary.merge([train_summary, test_summary]).eval(session=sess), step)
                    summary_writer.flush()

                    # Print out batch evaluation metrics
                    print("Iteration: {} train acc: {:.4f} test acc: {:.4f}".format(iteration, 
                                                                                    acc_train, 
                                                                                    acc_test))
        # Ensure file_writers closed
        summary_writer.close() 

In [224]:
tf.reset_default_graph()
wad = wide_and_deep(data_importer.train, data_importer.test, data_importer.num_reviewers, data_importer.num_products)

In [225]:
wad.build_model([1024, 512, 256], tf.nn.relu, 128, tf.nn.relu)

In [226]:
wad.add_optimizer(0.001)

In [227]:
sess = tf.Session()
wad.train_model(sess, 10, 100)
sess.close()


Starting epoch 1, running for 326 iterations
Printing evaluation metrics every 100 iterations

Iteration: 100 train acc: 0.0200 test acc: 0.0455
Iteration: 200 train acc: 0.0300 test acc: 0.0455
Iteration: 300 train acc: 0.0400 test acc: 0.0455

Starting epoch 2, running for 326 iterations
Printing evaluation metrics every 100 iterations

Iteration: 100 train acc: 0.0800 test acc: 0.0455
Iteration: 200 train acc: 0.0300 test acc: 0.0455
Iteration: 300 train acc: 0.0400 test acc: 0.0455

Starting epoch 3, running for 326 iterations
Printing evaluation metrics every 100 iterations

Iteration: 100 train acc: 0.0800 test acc: 0.0455
Iteration: 200 train acc: 0.0500 test acc: 0.0455
Iteration: 300 train acc: 0.1000 test acc: 0.0455

Starting epoch 4, running for 326 iterations
Printing evaluation metrics every 100 iterations

Iteration: 100 train acc: 0.0300 test acc: 0.0455
Iteration: 200 train acc: 0.0500 test acc: 0.0455
Iteration: 300 train acc: 0.0600 test acc: 0.0455

Starting epoch 